<a href="https://colab.research.google.com/github/komaltp/project/blob/main/English_Grammar_correction_Biderectional_word_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')
from tqdm import tqdm 
import tensorflow as tf
from  tensorflow.keras.preprocessing.sequence import pad_sequences
from  sklearn.model_selection import train_test_split
from tqdm import tqdm
import csv

In [ ]:
df= pd.read_csv("/content/new (1).csv")
df.columns = ["enc_input","dec_input","y"] 
df["dec_output"] = df.dec_input
df

,enc_input,dec_input,y,dec_output
0,I will be back soon.,I'll be back soon.,1.0,I'll be back soon.
1,I will be back soon.,I'll be back right away.,1.0,I'll be back right away.
2,You are in my way.,You're in my way.,1.0,You're in my way.
3,I may give up soon and just nap instead.,Maybe I'll just give up soon and take a nap in...,1.0,Maybe I'll just give up soon and take a nap in...
4,Are you sure?,Are you certain?,1.0,Are you certain?
...,...,...,...,...
248956,Please don't feed the trolls.,Please do not feed the trolls.,NaN,Please do not feed the trolls.
248957,Wanna make somethin' of it?,Wanna make something of it?,NaN,Wanna make something of it?
248958,Wanna make something of it?,Wanna make somethin' of it?,NaN,Wanna make somethin' of it?
248959,This won't go unpunished.,This will not go unpunished.,NaN,This will not go unpunished.


In [ ]:
df["dec_input"]= "<start> " + df["dec_input"]
df["dec_output"] =  df["dec_output"] + " <end>"

In [ ]:
df_sampled = pd.concat((df[df.y==1].sample(frac= 0.2,random_state=1),df[df.y==1]))
## ONCE THE DATA IS SAMPLED WE ARE SPLITTIND THE DATA IN TO TRAIN AND TEST

df_train ,df_val = train_test_split(df_sampled,test_size=0.2,random_state = 3, stratify = df_sampled.y )
## IN THE COLUMN WHICH HAS DECODER INPUTS ADDING "<end>" TOKEN TO BE LEARNED BY THE TOKENIZER

df_train["dec_input"].iloc[0]  = df_train.iloc[0]["dec_input"] + " <end>"

In [ ]:
df_sampled

,enc_input,dec_input,y,dec_output
9938,I do this too!,<start> I do this too.,1.0,I do this too. <end>
5045,They can't stop us.,<start> They cannot stop us.,1.0,They cannot stop us. <end>
45540,Tom told me he thought his house was haunted.,<start> Tom told me that he thought his house ...,1.0,Tom told me that he thought his house was haun...
44629,Tom told me that he'd be here by 2:30.,<start> Tom told me he'd be here by 2:30.,1.0,Tom told me he'd be here by 2:30. <end>
9583,Don't make fun of me.,<start> Don't make fun of me!,1.0,Don't make fun of me! <end>
...,...,...,...,...
46372,Tom thinks Mary isn't the one who did that.,<start> Tom thinks that Mary isn't the one who...,1.0,Tom thinks that Mary isn't the one who did tha...
46373,Tom thinks that Mary isn't the one who did that.,<start> Tom thinks Mary isn't the one who did ...,1.0,Tom thinks Mary isn't the one who did that. <end>
46374,Tom thinks Mary is the one who did that.,<start> Tom thinks that Mary is the one who di...,1.0,Tom thinks that Mary is the one who did that. ...
46375,Tom thinks that Mary is the one who did that.,<start> Tom thinks Mary is the one who did that.,1.0,Tom thinks Mary is the one who did that. <end>


In [ ]:
df_val

,enc_input,dec_input,y,dec_output
37551,Tom and Mary say they're not ready to do that ...,<start> Tom and Mary say they aren't ready to ...,1.0,Tom and Mary say they aren't ready to do that ...
17571,They're mine.,<start> They are mine!,1.0,They are mine! <end>
44299,Tom told me that I should get a little sleep.,<start> Tom told me I should get a little sleep.,1.0,Tom told me I should get a little sleep. <end>
12601,We are wasting time.,<start> We're wasting time.,1.0,We're wasting time. <end>
14385,I'm glad to see you!,<start> I am glad to see you!,1.0,I am glad to see you! <end>
...,...,...,...,...
30748,Tom will be very thirsty after working all day...,<start> Tom is going to be very thirsty after ...,1.0,Tom is going to be very thirsty after working ...
27506,Why aren't we allowed to do that here?,<start> Why are we not allowed to do that here?,1.0,Why are we not allowed to do that here? <end>
18382,That is my brother.,<start> That's my brother.,1.0,That's my brother. <end>
31465,Tom says he plans on going.,<start> Tom says he plans to go.,1.0,Tom says he plans to go. <end>


In [ ]:
np.random.seed(5) 
df_test = df.loc[np.random.choice(np.array([x for x in df.index.values if x not in df_sampled.index.values]),1000,replace= False,)]
df_test

,enc_input,dec_input,y,dec_output
62193,I suspect that Tom and Mary are worried.,<start> I suspect Tom and Mary are worried.,NaN,I suspect Tom and Mary are worried. <end>
217357,I don't think that anybody wants to do that.,<start> I don't think that anyone wants to do ...,NaN,I don't think that anyone wants to do that. <end>
61529,Tom's doctor advised him to stop drinking.,<start> Tom's doctor advised Tom to stop drink...,NaN,Tom's doctor advised Tom to stop drinking. <end>
222240,Tom said he wouldn't go anywhere because it wa...,<start> Tom said that he wouldn't go anywhere ...,NaN,Tom said that he wouldn't go anywhere because ...
146648,I don't think that anyone saw us doing that.,<start> I don't think that anybody saw us doin...,NaN,I don't think that anybody saw us doing that. ...
...,...,...,...,...
240950,"For your sake, I hope that doesn't happen.","<start> For your sake, I hope that that doesn'...",NaN,"For your sake, I hope that that doesn't happen..."
185106,I know that Tom knows Mary's not supposed to b...,<start> I know Tom knows Mary's not supposed t...,NaN,I know Tom knows Mary's not supposed to be doi...
62629,"I could have stayed a while longer, but I was ...","<start> I could've stayed a while longer, but ...",NaN,"I could've stayed a while longer, but I was ge..."
127619,Tom told me he hoped that Mary would be discreet.,<start> Tom told me that he hoped that Mary wo...,NaN,Tom told me that he hoped that Mary would be d...


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
## TOKENIZER FOR ENCODER INPUT
tk_inp = Tokenizer()
tk_inp.fit_on_texts(df_train.enc_input.apply(str))
# TOKENIZER FOR DECODER INPUT
tk_out = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n' )
tk_out.fit_on_texts(df_train.dec_input.apply(str))

In [ ]:
class Dataset :
    def __init__(self, data , tk_inp ,tk_out, max_len):
        self.encoder_inp = data["enc_input"].apply(str).values
        self.decoder_inp = data["dec_input"].apply(str).values
        self.decoder_out = data["dec_output"].apply(str).values
        self.tk_inp = tk_inp
        self.tk_out = tk_out
        self.max_len = max_len
        
    def __getitem__(self,i):
        # INPUT SEQUENCES
        self.encoder_seq = self.tk_inp.texts_to_sequences([self.encoder_inp[i]])
        # DECODER INPUT SEQUENCES 
        self.decoder_inp_seq = self.tk_out.texts_to_sequences([self.decoder_inp[i]])
        # DECODER INPUT SEQUENCES
        self.decoder_out_seq = self.tk_out.texts_to_sequences([self.decoder_out[i]])
        
        # PADDING THE ENCODER INPUT SEQUENCES
        self.encoder_seq = pad_sequences(self.encoder_seq, padding="post",maxlen = self.max_len)
        # PADDING THE DECODER INPUT SEQUENCES
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, padding="post",maxlen = self.max_len)
        # PADDING DECODER OUTPUT SEQUENCES
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq ,padding="post", maxlen = self.max_len)
        ##  RETURNING THE ENCODER INPUT , DECODER INPUT , AND DECODER OUTPUT
        return self.encoder_seq ,  self.decoder_inp_seq,  self.decoder_out_seq
    
    def __len__(self):
        # RETURN THE LEN OF INPUT ENDODER
        return len(self.encoder_inp)
## THIS CLASS CONVERTES THE DATASET INTO THE REQUIRED BATCH SIZE

class Dataloader(tf.keras.utils.Sequence):
    def __init__(self,batch_size,dataset):
        # INTIALIZING THE REQUIRED VARIABLES 
        self.dataset = dataset
        self.batch_size = batch_size
        self.totl_points = self.dataset.encoder_inp.shape[0]
        
    def __getitem__(self,i):
        # STATING THE START AND STOP VATIABLE CONTAINGING INDEX VALUES FOR EACH BATCH
        start = i * self.batch_size
        stop = (i+1)*self.batch_size
        
        # PLACEHOLDERS FOR BATCHED DATA
        batch_enc =[]
        batch_dec_input = []
        batch_dec_out =[]

        for j in range(start,stop): 
            
            a,b,c = self.dataset[j] 
            batch_enc.append(a[0]) 
            batch_dec_input.append(b[0])
            batch_dec_out.append(c[0]) 
        
        # Conveting list to array   
        batch_enc = (np.array(batch_enc)) 
        batch_dec_input = np.array(batch_dec_input)
        batch_dec_out = np.array(batch_dec_out)
        
        return [batch_enc , batch_dec_input],batch_dec_out
    
    def __len__(self):
        # Returning the number of batches
        return int(self.totl_points/self.batch_size)
#WE ARE TAKING THE MAXIMUM LENGHT EQUAL TO 35 WHICH IS 99 PERCENTILE OF THE WORD LENGTH DISTRUBUTION
# FORMING OBJECTS OF DATASET AND DATALOADER FOR TRAIN DATASET
train_dataset = Dataset(df_train,tk_inp,tk_out,35)
train_dataloader = Dataloader( batch_size = 512, dataset=train_dataset)
# FORMING OBJECTS OF DATASET AND DATALOADER FOR VALIDATION DATASET
val_dataset = Dataset(df_val , tk_inp,tk_out,35)
val_dataloader = Dataloader(batch_size=512 , dataset=val_dataset)

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model
## ENDOCER CLASS
class Encoder(tf.keras.layers.Layer):
    '''
    Encoder model -- That takes a input sequence and returns encoder-outputs,encoder_final_state_h,encoder_final_state_c
    '''
    def __init__(self , vocab_size , embedding_dim , enc_units , input_len):
        super().__init__()
        
        # STATING ALL THE VARIABLES
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.input_len = input_len
        self.enc_units = enc_units
        
        # INITALIZING EMBEDDING LAYER
        self.embedding = layers.Embedding(input_dim= self.vocab_size,
                                         output_dim = self.embedding_dim,
                                         mask_zero = True,
                                          input_length = self.input_len
                                         )
        # INTIALIZING LSTM LAYER
        self.lstm_bi = layers.Bidirectional(layers.LSTM(units= self.enc_units,return_state = True,return_sequences=True ))
        self.concat1 = layers.Concatenate()
        self.concat2 = layers.Concatenate()
    def call(self,input):
        '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to encoder_lstm
          returns -- encoder_output, last time step's hidden and cell state
        '''
        # CONVERTING INPUT TO EMBEDDED VECTORS
        emb = self.embedding(input)
        # PASSING THROUGH LSTM LAYER
        enc_output , state_h1 , state_c1 ,state_h2 , state_c2 = self.lstm_bi(emb)
        state_h = self.concat1([state_h1,state_h2])
        state_c  = self.concat2(([state_c1,state_c2]))

        return enc_output ,state_h ,state_c 

## DECODER CLASS
class Decoder(tf.keras.layers.Layer):
    def __init__(self,vocab_size , embedding_dim, dec_unit,input_len ):
        super().__init__()
        # INITALIZING ALL THE VARIABLES 
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.input_len = input_len
        self.dec_unit =dec_unit
        
        
    def build(self,input_shape):
        
        # INITALIZING EMBEDDING AND LSTM LAYER
        self.embedding = layers.Embedding(input_dim = self.vocab_size,
                                          output_dim = self.embedding_dim,
                                         mask_zero=True,
                                         input_length = self.input_len)
        self.lstm = layers.LSTM(units = self.dec_unit,
                               return_sequences=True,
                               return_state=True)
        
    def call(self,input, state):
        # FORMING THE EMBEDDED VECTORS
        emb = self.embedding(input)
        
        # LSTM OUTPUT
        dec_out,state_h,state_c = self.lstm(emb,initial_state = state)
        
        return dec_out,state_h,state_c
  
## DEFINING THE ARCHITECTURE

# INPUT LAYER
enc_input = layers.Input(shape=(35))
enc_output,state_h,state_c =   Encoder(vocab_size= len(tk_inp.word_index)+1  , embedding_dim= 300 ,
                               enc_units=256 ,input_len=35)(enc_input)
## STORING ENCOER STATES IN A VARIABLE
enc_state = [state_h,state_c] 

# INPUT LAYER FOR DECODER
dec_input = layers.Input(shape = (35))
## DECODER LAYER
dec_output,_,_ = Decoder(vocab_size = len(tk_out.word_index)+1  , embedding_dim = 300,
                               dec_unit=512,input_len=35)(dec_input,enc_state)

# DENSE LAYER
dense = layers.Dense(len(tk_out.word_index)+1,activation="softmax")(dec_output)

# MODEL DEFINING
model  = Model(inputs=[enc_input,dec_input],outputs=dense)        

In [ ]:
callback =[ tf.keras.callbacks.ModelCheckpoint( "/content/drive/MyDrive/Colab Notebooks/cs2/model_save/bidirectional_train_emb/besh.h5",save_best_only=True,mode="min" ,save_weights_only=True),
           tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,verbose=1,min_delta=0.0001),
            tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/Colab Notebooks/cs2/model_save/bidirectional_train_emb/logs/save",histogram_freq=1)
]
## TRAINNG AND VALIDATION STEPS FOR ONE EPOCH
train_steps = train_dataloader.__len__()
val_steps  = val_dataloader.__len__()

# COMPILING THE MODEL
model.compile(optimizer="adam",loss='sparse_categorical_crossentropy')
## MODEL SUMMARY
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 35)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 35)]         0           []                               
                                                                                                  
 encoder (Encoder)              ((None, 35, 512),    3106636     ['input_1[0][0]']                
                                 (None, 512),                                                     
                                 (None, 512))                                                     
                                                                                              

In [ ]:
history = model.fit(train_dataloader, steps_per_epoch=train_steps,epochs= 10,validation_data = val_dataloader,validation_steps =val_steps,callbacks=callback)

Layer Encoder has arguments ['self', 'vocab_size', 'embedding_dim', 'enc_units', 'input_len']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config


Epoch 1/10
86/86 [==============================] - 1411s 16s/step - loss: 1.3231 - val_loss: 1.1078
Epoch 2/10
86/86 [==============================] - 1343s 16s/step - loss: 1.0139 - val_loss: 0.8957
Epoch 3/10
86/86 [==============================] - 1314s 15s/step - loss: 0.8272 - val_loss: 0.7574
Epoch 4/10
86/86 [==============================] - 1321s 15s/step - loss: 0.7081 - val_loss: 0.6630
Epoch 5/10
86/86 [==============================] - 1340s 16s/step - loss: 0.6167 - val_loss: 0.5867
Epoch 6/10
86/86 [==============================] - 1355s 16s/step - loss: 0.5359 - val_loss: 0.5146
Epoch 7/10
86/86 [==============================] - 1358s 16s/step - loss: 0.4614 - val_loss: 0.4524
Epoch 8/10
86/86 [==============================] - 1376s 16s/step - loss: 0.3974 - val_loss: 0.4019
Epoch 9/10
86/86 [==============================] - 1386s 16s/step - loss: 0.3413 - val_loss: 0.3589
Epoch 10/10
86/86 [==============================] - 1430s 17s/step - loss: 0.2945 - val_lo

In [ ]:
def predict(ita_text,model):
    
    # forming integer sequences
    seq = tk_inp.texts_to_sequences([ita_text])
    # padding the sequences
    seq = pad_sequences(seq,maxlen = 20 , padding="post")
    
    # generating the output from encoder
    enc_output,state_h,state_c= model.layers[2](seq)
    
    # placeholder for predicted output
    pred = []
    
    input_state = [state_h,state_c]
    # initailizing the vector for inputing to decoder
    current_vec = tf.ones((1,1))
    
    for i in range(20): # for each word in the input
        # passing each word through decoder layer
        dec_output,dec_state_h,dec_state_c = model.layers[3](current_vec , input_state)
        # passing decoder output through dense  layer
        dense = model.layers[4](dec_output)
        # taking argmax and getting the word index and updating the current vector
        current_vec = np.argmax(dense ,axis = -1)
        # updating the decoder states
        input_state = [dec_state_h,dec_state_c]
        # getting the actual word from the vocab
        pred.append(tk_out.index_word[current_vec[0][0]])
        
        # if the actual word is <end> break the loop
        if tk_out.index_word[current_vec[0][0]]=="<end>":
            break
        
    return " ".join(pred)

In [ ]:
import nltk.translate.bleu_score as bleu
# BELU SCORE
BLEU = []
np.random.seed(1)
test_data = df_val.loc[np.random.choice(df_val.index,size = 100,replace=False)]
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        pred = predict(str(i.enc_input),model).split()
        act = [str(i.dec_output).split()]
        b = bleu.sentence_bleu(act,pred)
        #print(b)
        BLEU.append(b)
        
    except:
      continue
print(BLEU)
print("BELU = ", np.mean(BLEU))

104it [00:19,  5.32it/s]

[0.2891784933232572, 0.2846946938149361, 8.38826642100846e-155, 9.283142785759642e-155, 9.283142785759642e-155, 3.845723017268071e-78, 0.18349818317455793, 1.4488496539373276e-231, 5.886417212929789e-155, 7.711523862191631e-155, 1.2183324802375697e-231, 1.384292958842266e-231, 0.5410822690539396, 1.4740564900137075e-231, 1.331960397810445e-231, 3.661843723291765e-78, 1.5319719891192393e-231, 0.6989307622784944, 0.22131477988685874, 1.384292958842266e-231, 1.2882297539194154e-231, 1.5319719891192393e-231, 0.3303164318013807, 0.3303164318013807, 6.227170448085464e-155, 9.283142785759642e-155, 0.4671379777282001, 1.1931009847695213e-231, 1.4256605770826504e-231, 6.8489908526642754e-155, 1.331960397810445e-231, 1.2882297539194154e-231, 1.1640469867513693e-231, 1.1640469867513693e-231, 0.42728700639623407, 1.384292958842266e-231, 9.853445011990208e-232, 1.5319719891192393e-231, 9.283142785759642e-155, 1.2917956969975423e-231, 5.072841446586652e-78, 5.795035077445241e-155, 0.3655552228545123

In [ ]:
def beam_search(input,model,k):
    seq = tk_inp.texts_to_sequences([input])
    seq = pad_sequences(seq,maxlen = 35,padding="post")

    
    enc_gru ,enc_state_h,enc_state_c   = model.layers[2](seq)

    input_state = [enc_state_h,enc_state_c ]
    
    k_beams = [[tf.ones((1,1),dtype=tf.int32),0.0]]
    for i in range(35):
        candidates = []
        for sent_pred , prob in k_beams :
            if tk_out.word_index["<end>"] in sent_pred.numpy() :

                candidates += [[sent_pred , prob]]
            else:
                
                dec_gru , dec_state_h ,dec_state_c = model.layers[3](sent_pred , input_state)
                dense = model.layers[4](tf.expand_dims(dec_state_h,axis=0))
                pred = tf.argsort(dense, direction= 'DESCENDING')[:,:,:k]
                for w in range(k):
                  candidates += [[tf.concat((sent_pred, pred[:,:,w]) , axis=-1) , (prob + tf.math.log(dense[:,:,pred[:,:,w][0][0]])[0][0])]  ]
        k_beams = sorted(candidates,key=lambda tup:tup[1],reverse=True)[:k]

    all_sent = []
    for i,score in k_beams:
        sent = ""
        for j in range(1,35):
            sent +=  tk_out.index_word[i.numpy()[:,j][0]] +  " " 
            if tk_out.index_word[i.numpy()[:,j][0]] =="<end>":
                break
        all_sent.append((sent.strip(),score.numpy()))
    return all_sent
# VALIDATION BELU SCORE
BLEU_beam = []
np.random.seed(1)
test_data = df_val.loc[np.random.choice(df_val.index,size = 100,replace=False)]
for ind,i in tqdm(test_data.iterrows(),position=0):

    try:
        pred = beam_search(str(i.enc_input),model,3)[0][0].split()
        act = [str(i.dec_output).split()]
        b =bleu.sentence_bleu(act,pred)
        BLEU_beam.append(b)
    except:
          continue

print("BELU Score = ",np.mean(BLEU_beam)) 

104it [01:18,  1.33it/s]

BELU Score =  0.11441641153698286


In [ ]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[95])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[95],model))
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[95])

INPUT SENTENCE ===>  I thought Tom would get mad for sure.
PREDICTED SENTENCE ===>  i thought tom would get angry for him <end>
ACTUAL SENTENCE ===>  I thought that Tom would get mad for sure. <end>
